Lien modèles implémentés : https://huggingface.co/

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
path = '../Semantic_chunk_identification'
os.chdir(path)
%ls

 Le volume dans le lecteur E s'appelle DATA
 Le num‚ro de s‚rie du volume est C61F-A303

 R‚pertoire de E:\timot\Documents\1 - Centrale Marseille\0.6 - Semestre S9\0_PSTALN\TP\Semantic_chunk_identification

11/01/2022  18:18    <DIR>          .
11/01/2022  18:18    <DIR>          ..
02/01/2022  17:34               447 .gitignore
30/12/2021  15:12    <DIR>          .ipynb_checkpoints
26/12/2021  20:43           721ÿ719 annotation_guidelines_interpretable_sts2016v2.2.pdf
28/12/2021  16:43         4ÿ942ÿ553 Article Inspire 2016.pdf
01/12/2021  12:51           249ÿ099 projet.pdf
02/01/2022  17:26           119ÿ919 PROJ-PSTALN Baselines.ipynb
11/01/2022  18:18           236ÿ188 PSTALN - Model state of the art.ipynb
09/01/2022  10:42    <DIR>          rapport
17/12/2021  15:21                78 README.md
17/12/2021  15:29                25 requirements.txt
31/12/2021  17:59    <DIR>          test_goldStandard
23/10/2015  14:49    <DIR>          train_2015_10_22.utf-8
31/12/2021  17:59    <DI

In [5]:
import re #Regular expressions operation

def readFile(filename):
    return [line.strip() for line in open(filename)]
#.decode("utf-8")

def readFileWithChunks(filename):
    lines_marked = [line.strip() for line in open(filename)]
    # .decode("utf-8")
    lines_clean1 = [line.replace('[ ', "") for line in lines_marked]
    lines_clean2 = [line.replace(' ]', "") for line in lines_clean1]
    return lines_clean2

In [7]:
path_headline1_chunked = 'train_2015_10_22.utf-8/STSint.input.headlines.sent1.chunk.txt'
path_headline2_chunked = 'train_2015_10_22.utf-8/STSint.input.headlines.sent2.chunk.txt'
path_headline1 = 'train_2015_10_22.utf-8/STSint.input.headlines.sent1.txt'
path_headline2 = 'train_2015_10_22.utf-8/STSint.input.headlines.sent2.txt'

lines_chunked = True  #False

if lines_chunked == True:
    sent1_lines = readFile(path_headline1)
    sent2_lines = readFile(path_headline2)
    sent1_lines_chunked = readFile(path_headline1_chunked)
    sent2_lines_chunked = readFile(path_headline2_chunked)
else:
    sent1_lines = readFile(path_headline1)
    sent2_lines = readFile(path_headline2)

if len(sent1_lines) != len(sent2_lines):
    print ("files have different number of lines... quiting")
    exit(-1)

In [8]:
idx = 3
print(sent1_lines[idx])
print(sent1_lines_chunked[idx])

Syria peace plan conditions " unacceptable , " opposition says
[ Syria peace plan conditions ] [ " ] [ unacceptable ] [ , ] [ " ] [ opposition ] [ says ]


In [9]:
length = len(sent1_lines)
trainSamples = sent1_lines[:int(3/4*length)]
trainSamplesChunked = sent1_lines_chunked[:int(3/4*length)]

testSamples = sent1_lines[int(3/4*length):]
testSamplesChunked = sent1_lines_chunked[int(3/4*length):]

#TODO : Ajouter fichier 2 et données de test

print('Train size:', len(trainSamples))
print('Test size:', len(testSamples))

Train size: 567
Test size: 189


##Tokeniser, Model & Chunking

> Load from internet (Hugging Face <3 ) 

In [11]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def get_chunk_entity(sentence):
  tokenizer = AutoTokenizer.from_pretrained("vblagoje/bert-english-uncased-finetuned-chunk")
  model = AutoModelForTokenClassification.from_pretrained("vblagoje/bert-english-uncased-finetuned-chunk")
  pipe = pipeline('ner', grouped_entities=True, model=model, tokenizer=tokenizer)
  chunk = pipe(sentence)
  nombre_entites = len(chunk)
  liste_chunck = []
  for i in range(nombre_entites):
    entity = chunk[i]['word']
    entity_group = chunk[i]['entity_group']
    liste_chunck.append([entity,entity_group])
  return liste_chunck

def get_comparables(chunk1, chunked_sentence2):
  entity = chunk1[1]
  longueur_sequence = len(chunked_sentence2)
  liste_chunck_comparables = []
  for i in range(longueur_sequence):
    if chunked_sentence2[i][1] == entity:
      liste_chunck_comparables.append(chunked_sentence2[i])
  return liste_chunck_comparables

def my_cosine_similarity(chunk1, liste_chunck_comparables):
  nombre_chunk_comparables = len(liste_chunck_comparables)
  sentences = [chunk1[0]]
  sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
  for i in range(nombre_chunk_comparables):
    print(liste_chunck_comparables[i][0])
    sentences.append(liste_chunck_comparables[i][0])
  sentence_embeddings = sbert_model.encode(sentences)
  cos_sim = cosine_similarity(sentence_embeddings)
  return cos_sim

In [12]:
sentence1 = 'Bangladesh building disaster death toll passes 500'
sentence2 = 'Barack Obama lives in New York'
sentence3 = 'John Demjanjuk, convicted Nazi death camp guard, dies aged 91'

#True chunked line : [ Saudi Women ] [ Allowed ] [ To Compete ] [ At Olympics ]
#Predicted chunked line : 	 [ Saudi Women ] Allowed [ To Compete ] [ At Olympics ]

#True chunked line : 		 [ Drone strike ] [ kills ] [ four suspected militants ] [ in Pakistan ]
#Predicted chunked line : 	 [ Drone strike ] [ kills ] [ four ] [ suspected ] [ militants ] [ in Pakistan ]

my_liste1 = get_chunk_entity(sentence1)
my_liste2 = get_chunk_entity(sentence2)
my_liste3 = get_chunk_entity(sentence3)

liste_chunck_comparables = get_comparables(my_liste1[0], my_liste2)
my_cos = my_cosine_similarity(my_liste1[0], liste_chunck_comparables)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

barack obama
new york


In [7]:
my_liste3

[['john demjanjuk', 'NP'],
 ['convicted nazi death camp guard', 'NP'],
 ['dies', 'VP'],
 ['aged', 'NP'],
 ['91', 'NP']]

##Chunking

> Voir Embeddings déja entrainé inclus. ?

Lien : https://huggingface.co/vblagoje/bert-english-uncased-finetuned-chunk?text=My+name+is+Wolfgang+and+I+live+in+Berlin

Lien POS tagging pour l'utiliser: https://stackoverflow.com/questions/65419656/parsing-the-hugging-face-transformer-output (ne chunk pas comme on veut )
 

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("vblagoje/bert-english-uncased-finetuned-chunk")
model

#TODO : Finetuning https://medium.com/@andrewmarmon/fine-tuned-named-entity-recognition-with-hugging-face-bert-d51d4cb3d7b5

In [ ]:
pipe = pipeline('ner', grouped_entities=True, model=model, tokenizer=tokenizer)

trainChunking = pipe(trainSamples)

##Choses à réfléchir 🇰
- Loss functions (ex pas BLEU car on ne génère pas des séquences)
- Optimizers ?

https://bamblebam.medium.com/generating-text-similarity-scores-using-bert-44a78df1f9c7

In [ ]:
# Test Timothée : ça fonctionne !
#pipe(trainSamples)